In [14]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run headless Chrome, optional
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

In [15]:
# !pip3 install selenium

In [16]:
"""
TimeOut LA
Discover LA
WE Like LA
Secret Los Angeles
KCRW Events
"""

'\nTimeOut LA\nDiscover LA\nWE Like LA\nSecret Los Angeles\nKCRW Events\n'

In [23]:
def load_more_events2(driver):
    """
    This loading function
    jumps down to the viewMore div and moves slightly up the screen,
    which seems to work for timeout
    """
    for _ in range(20): # only try 20 times for now
        try:
            time.sleep(5)
            element = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "div[class*='viewMore']"))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", element)
            driver.execute_script(f"window.scrollBy(0, -100);")
        except:
            print("can't scoll anymore")
            break
    return(True)
    
# def load_more_events1(driver):
#     """
#     this old loading function
#     moved slowly down the page
#     because jumping down caused problems
#     """
#     # Function to scroll down the page
#     def scroll_down(driver):
#         # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#          driver.execute_script("window.scrollBy(0, 500);")
#          new_height = driver.execute_script("return document.body.scrollHeight")
#          return(new_height)


#     # Scroll 60% of the way to start
#     total_height = driver.execute_script("return document.body.scrollHeight")
#     scroll_height = total_height * 0.40
#     driver.execute_script(f"window.scrollBy(0, {scroll_height});")
#     time.sleep(3)
#     height = total_height

#     # Initialize a list to store the article URLs
#     article_urls = list()
    
#     # Scroll and collect article URLs
#     SCROLL_PAUSE_TIME = 1
#     max_attempts = 50  # Maximum number of scrolls
#     attempts = 0
#     num_hrefs = 0

#     # Get all the events 
#     while True:
#         # Scroll down the page
#         """
#         This strange scrolling + checking issue
#         Is because I can't find a good way to get the 
#         feed to continually refresh as the bot
#         scrolls down the page -- if I scroll too fast
#         it doesn't update and we get stuck on the footer
#         and don't refresh, too slow and it takes forever
#         to run.
    
#         Currently it moves down 500px, and checks 
#         every 50 scrolls if new content has emerged.
#         I'm sure there is a way to simple add a conditional
#         but MOVING ON.
#         """
#         new_height = scroll_down(driver)
#         # print("#", attempts)
        
#         # Wait to load the page
#         time.sleep(SCROLL_PAUSE_TIME)
        
#         # Only check every 50 scrolls
#         attempts += 1
#         if attempts % 50 != 0:
#             continue
#         # Collect new titles
#         else:
#             ## THIS WILL BREAK NOW:
#             added, article_urls = collect_titles(article_urls, driver)
#             if added == 0:
#                 break
    
#     print(len(article_urls), 'total articles found.')

#     return(True)


    
def collect_titles(driver):
    # Get articles on the page
    title_divs = driver.find_elements(By.XPATH, '//*[contains(@class, "title")]')
    article_urls = []

    # Collect URLs from within these divs
    for title in title_divs:
        links = title.find_elements(By.TAG_NAME, 'a')
        for link in links:
            href = link.get_attribute('href')
            if href and 'timeout.com/los-angeles/' in href:
                if href not in article_urls:
                    article_urls.append(href)
    print(len(article_urls), "new articles")

    return(article_urls)

def pull_tout_event_info(driver, article_urls):
    
    event_title = []
    event_description = []
    event_date = []
    event_time = []
    event_location = []
    event_price = []
    event_urls = []
    
    for event_url in article_urls:
        # print(event_url)
        driver.get(event_url)
        time.sleep(2)
    
        title = event_url.split('/')[-1].replace("-"," ").title()
        # event_title.append(title)
        
        try:
            details_sections = driver.find_element(By.CSS_SELECTOR, 'section[data-section-name="review"]')
            descr_txt = details_sections.text
        except:
            descr_txt = "None."
        try:
            occurrence_section = driver.find_element(By.CSS_SELECTOR, 'section[data-section-name="occurrences"]')
            zones = occurrence_section.find_element(By.CLASS_NAME, 'zoneItems')
            child_divs = zones.find_elements(By.XPATH, './div')
            timestamps = []
            venues = []
            prices = []
            for entry in child_divs:
                time_element = entry.find_element(By.TAG_NAME, 'time')
                timestamp = time_element.get_attribute('datetime') 
                timestamps.append(timestamp)
    
                try:
                    venue = entry.find_element(By.XPATH, './/span[contains(@class, "venueName")]/a').text
                    venues.append(venue)
                except:
                    venues.append("unknown")
    
                try:
                    price = entry.find_element(By.XPATH, './/div[contains(@class, "price")]').text
                    prices.append(price)
                except:
                    prices.append("unknown")
        except:
            timestamps = ["unknown"]; venues = ["unknown"]; prices = ["unknown"]
            
            
        for i in range(len(timestamps)):
            if len(timestamps) > 1:
                event_title.append(title+f" (Day {i+1}")
            else:
                event_title.append(title)
            event_date.append(timestamps[i])
            event_time.append(timestamps[i])
            event_location.append(venues[i])
            event_price.append(prices[i])
            event_description.append(descr_txt)
            event_urls.append(event_url)

    p = pd.DataFrame({"Name":event_title, 
                      "Description":event_description,
                      "Date":event_date,
                      "Time":event_time,
                      "Location":event_location,
                      "Price": event_price,
                      "URL":event_urls})
    return(p)

def pull_timeout_calendar():
    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome(options=chrome_options)
    
    months = ['august','september','october','november','december']
    
    events_pd = pd.DataFrame({'Name':[],
                              'Description':[],
                              'Date': [],
                              'Time':[],
                              'Location':[],
                              'Price':[],
                              'URL':[]})
    for month in months:
        print(month)
    
        # Navigate to the Time Out Los Angeles page
        url = f"https://www.timeout.com/los-angeles/things-to-do/{month}-events-calendar"
        try:
            driver.get(url)
        except:
            continue
    
        load_more_events2(driver)
        
        article_urls = collect_titles(driver)
        
        p = pull_tout_event_info(driver, article_urls)
    
        events_pd = pd.concat([events_pd,p])

    driver.quit()

    return(p)

In [24]:
## Discover Los Angeles
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_links(driver):
    return(
        driver.find_elements(By.XPATH, "//a[not(contains(@style, 'display:none')) and not(contains(@style, 'visibility:hidden'))]")
    )
def load_more_days(driver):
    for _ in range(5): # only try 5 times for now
        try:
            time.sleep(3)
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//*[contains(@id, 'load-more')]"))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", element)
            driver.execute_script(f"window.scrollBy(0, -100);")
            element.click()
        except:
            print("can't load anymore.")
            break
        try:
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "img[name^='ajax-loader']")))
            # Takes a long time to load...
            element = WebDriverWait(driver, 1000).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, "img[name^='ajax-loader']")))
            print("The loading GIF has disappeared.")
        except:
            print("The loading GIF is still visible or the timeout was reached.")
            break
    return(True)

def load_more_daily_events(driver):
    anchors = get_links(driver)
    print("links before:", len(anchors))
    time.sleep(3)
    elements = driver.find_elements(By.CSS_SELECTOR, "ul")
    for e in elements:
        if 'item' in e.text:
            item_element = e.find_element(By.XPATH, ".//*[contains(text(), 'item')]")
            driver.execute_script("arguments[0].click();", item_element)
            time.sleep(1)
    anchors = get_links(driver)
    print("links after:",len(links))

    return(True)

def collect_links(driver):
    anchors = get_links(driver)
    visited_links = list(set(visited_links))
    c = 0
    visited_links = []
    for a in anchors:
        if c % 100 == 0:
            print(c)
        c += 1
        link = a.get_attribute('href')
        
        if not link:
            continue
        elif ('https://www.discoverlosangeles.com/event/' not in link) or link in visited_links:
            continue
        else:
            visited_links.append(link)
    print(len(visited_links)," events found.")

    return(visited_links)

def collect_event_info(driver,visited_links):

    name_list = []
    price_list = []
    date_list = []
    time_list = []
    location_list = []
    desc_list = []
    for link in visited_links:
        name = link.split('/')[-1].replace('-',' ').strip().title()
        # print(link)
        # print(name)
        driver.get(link)
    
        price = None
        date = None
        time_ = None
    
        ## Get event details
        try:
            info = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div[class*='dla-information']"))
        )
            info = info.text
            info = ' '.join(info.split()).strip()
    
            for fact in info.split("|"):
                if '$' in fact:
                    price = '$'+fact.split("$")[-1]
                elif 'PM' in fact or 'AM' in fact:
                    time_ = ' '.join(fact.split())
                elif '2024' in fact:
                    date = ' '.join(fact.split())
                    ## TODO: There are date ranges in here!
                    if "-" in date:
                        date = " ".join(date.split("-")[0].split())
                    else:
                        date = date.split(", 2024")[0]
            # print("Price:", price)
            # print("Date:", date)
            # print("Time:", time)
        except:
            # print("no info")
            info = None
    
        ## Get location
        try:
            location = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "span[class*='dla-venue']"))
        )
            location = location.text
            location = ' '.join(location.split()).strip()
            # print("LOCATION: ",location)
        except:
            # print("no location")
            location = None
    
        try:
            description = driver.find_element(By.XPATH, "//div[@class='dla-txt']")
            # description = location.text
            # description = ' '.join(description.split()).strip()
            description = description.text
            # print("DESC: ",description)
        except:
            # print("no desc")
            description = None
    
        name_list.append(name)
        price_list.append(price)
        date_list.append(date)
        time_list.append(time_)
        location_list.append(location)
        desc_list.append(description)
    
    p = pd.DataFrame({
        "Name":name_list,
        "Date":date_list,
        "Description":desc_list,
        "Time":time_list,
        "Location":location_list,
        "Price":price_list,
        "URL":visited_links
    })
    return(p)

def pull_dla_calendar():
    url = 'https://www.discoverlosangeles.com/events'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    
    load_more_days(driver)
    load_more_daily_events(driver)
    visited_links = collect_links(driver)
    p = collect_event_info(driver, visited_links)
    
    driver.quit()

    return(p)

In [ ]:
print("timeout...")
p1 = pull_timeout_calendar()
print("discovery la...")
p2 = pull_dla_calendar()

timeout...
august
can't scoll anymore
95 new articles
september
can't scoll anymore
64 new articles
